# 교차 검증과 그리드 서치

## 검증 세트

In [33]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [34]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [35]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [36]:
# 책에는 0.2던데 0.8*0.25 = 0.2이기 때문에 0.25로 변경
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.25, random_state=42)

In [37]:
print(sub_input.shape, val_input.shape)

(3897, 3) (1300, 3)


In [38]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9969207082371054
0.86


## 교차 검증

In [39]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00455499, 0.00430107, 0.00448585, 0.00449991, 0.00439692]), 'score_time': array([0.00046611, 0.00046778, 0.00054193, 0.0004611 , 0.00053   ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [40]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [41]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [42]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

In [43]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [44]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) # -1은 가능한 많은 코어 사용

In [45]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [46]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [47]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [48]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [49]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [50]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [51]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [52]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [53]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

In [54]:
from scipy.stats import uniform, randint

In [55]:
rgen = randint(0, 10)
rgen.rvs(10)

array([9, 2, 5, 9, 1, 6, 7, 0, 8, 8])

In [56]:
np.unique(rgen.rvs(1000), return_counts=True)
# return_count는 앞선 배열의 값들이 각 몇번씩 나왔는지 체크

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([124, 107, 110, 109,  90,  89,  94, 100,  79,  98]))

In [57]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.22899044, 0.87055265, 0.17181637, 0.92453844, 0.83833033,
       0.44718902, 0.51443104, 0.94648666, 0.35122372, 0.99707533])

In [58]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [59]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1755cbc40>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1755cb760>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13015e430>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1755d0370>},
                   random_state=42)

In [60]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [61]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [62]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제 풀어보기

In [63]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1755cbc40>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x1755cb760>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x13015e430>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x1755d0370>},
                   random_state=42)

In [64]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077
